In [3]:
import tweepy
import time
import pandas as pd
 
# assign the values accordingly
consumer_key = "vILOcHxSRk9zOy5fZH3x7kIjS"
consumer_secret = "VzPDu5BEq3x8VglxJPGKa1lFKfJ5OoKZfd90KOES1mdtMQsezo"
access_token = "1526192300182978562-iHYctxjSI0k1cXZGIihYu6jQQJYZpF"
access_token_secret = "zSwpEAjQwpDFL6jzhSrZZTf3spepUAxzTbvwy8TfNT1g9"

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
 
# set access to user's access key and access secret
auth.set_access_token(access_token, access_token_secret)
 
# calling the api
api = tweepy.API(auth, wait_on_rate_limit=True)

client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAJm%2BcgEAAAAAdT6oZA5nHHlM%2BU07cRioA8QSXvI%3D1ZN1biqO1uHYO4GxiOEpSXCTElUu7fz7fjk8GTmEus6DJBYp2G',wait_on_rate_limit=True)

    
retweets_list=[]
like_list=[]
user_listlike=[]
user_listRT=[]

tweet_id="1518972397277396992"

    #get received retweets
for user in tweepy.Paginator(client.get_retweeters,id=tweet_id, max_results=100).flatten(limit=100): 
    retweets_list.append(user.username)
time.sleep(20)
    #get received likes
for user in tweepy.Paginator(client.get_liking_users,id=tweet_id, max_results=100).flatten(limit=100):
    like_list.append(user.username)
time.sleep(20)

        
    

        

retweets_list = list(set(retweets_list))
like_list=list(set(like_list))
dfRT = pd.DataFrame(retweets_list, columns=['Username'])
dfL= pd.DataFrame(like_list, columns=['Username'])
#dfRT.to_csv('C:/Users/srabh/Documents/PFEcode/SSTRT.csv')
#dfL.to_csv('C:/Users/srabh/Documents/PFEcode/SSTLikes.csv')

In [4]:
#telecharger entre 2 dates avant le Newom (2 semaines avant) les tweets du mec declencheur
#regarder si dans la liste qu'on a telecharger si elles ont interagit(liker,RT) les tweets des 2 semaines d'avant

SST_list=[]

In [25]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time


os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAJm%2BcgEAAAAAdT6oZA5nHHlM%2BU07cRioA8QSXvI%3D1ZN1biqO1uHYO4GxiOEpSXCTElUu7fz7fjk8GTmEus6DJBYp2G'

def auth(): 
    return os.getenv('TOKEN')


def create_headers(bearer_token): 
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

#def connect_to_endpoint(url, headers, params, next_token = None):
    #params['next_token'] = next_token   #params object received from create_url function
    #response = requests.request("GET", url, headers = headers, params = params)
    #print("Endpoint Response Code: " + str(response.status_code))
    #if response.status_code != 200:
        #raise Exception(response.status_code, response.text)
    #return response.json()

def connect_to_endpoint(url, headers, params, next_token = None):
    headers = create_headers(auth())
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def append_to_csv(json_response, fileName):

    

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']#['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']
        
        if ('in_reply_to_user_id' in tweet):
            in_reply_to_user_id = tweet['in_reply_to_user_id']
        else:
            in_reply_to_user_id = " "
        
        

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']
        
        #test
        conversation_id= tweet['conversation_id']
        
      


        # 7. source
        #source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count,conversation_id,in_reply_to_user_id,text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 
    
# append_to_csv(json_response, "data.csv") 

def tweets_to_dataframe(json_response):
    tweets = []
    for tweet in json_response['data']:
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']#['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']
        
        if ('in_reply_to_user_id' in tweet):
            in_reply_to_user_id = tweet['in_reply_to_user_id']
        else:
            in_reply_to_user_id = " "
        

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']
        
        #test
        conversation_id= tweet['conversation_id']
        

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a dictionary
        tweets.append({'author_id': author_id, 'created_at': created_at, 'geo': geo, 'tweet_id': tweet_id, 'lang': lang, 'like_count': like_count, 'quote_count': quote_count, 'reply_count': reply_count, 'retweet_count': retweet_count,'conversation_id':conversation_id,'in_reply_to_user_id':in_reply_to_user_id,'text':text})
    # Return the tweets as a dataframe
    return pd.DataFrame(tweets)
    
bearer_token='AAAAAAAAAAAAAAAAAAAAAJm%2BcgEAAAAAdT6oZA5nHHlM%2BU07cRioA8QSXvI%3D1ZN1biqO1uHYO4GxiOEpSXCTElUu7fz7fjk8GTmEus6DJBYp2G'
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword=["from:819884626328121348"]
start_list=["2022-03-01T00:00:00.000Z"]
end_list=["2022-04-03T00:00:00.000Z"]
lang='en'
max_results = 500

total_tweets = 0

# Create file
csvFile = open("SSTdelta.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','conversation_id','in_reply_to_user_id','tweet'])
csvFile.close()
for j in range(len(keyword)):
    res=keyword[j]
    for i in range(0,len(start_list)):

        num_file=str(j)

        # Inputs
        count = 0 # Counting tweets per time period
        max_count = 2000000 # Max tweets per time period
        flag = True
        next_token = None
        # Check if flag is true
        while flag:
            # Check if max_count reached
            if count >= max_count:
                break
            print("-------------------")
            print("Token: ", next_token)
            url = create_url(res, start_list[i],end_list[i], max_results)
            json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
            result_count = json_response['meta']['result_count']

            if 'next_token' in json_response['meta']:
                # Save the token to use for next call
                next_token = json_response['meta']['next_token']
                print("Next Token: ", next_token)
                if result_count is not None and result_count > 0 and next_token is not None:
                    print("Start Date: ", start_list[i])
                    append_to_csv(json_response, "C:/Users/srabh/Documents/PFEcode/SSTdelta.csv")
                    count += result_count
                    total_tweets += result_count
                    print("Total # of Tweets added: ", total_tweets)
                    print("-------------------")
                    time.sleep(5)                
            # If no next token exists
            else:
                if result_count is not None and result_count > 0:
                    print("-------------------")
                    print("Start Date: ", start_list[i])
                    #name_file="test_"+num_file+".csv"
                    df=tweets_to_dataframe(json_response)
                    auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
                    api = tweepy.API(auth, wait_on_rate_limit=True)
                    counter=0
                    for k in df.iloc[:,3]:
                        tweet = api.get_status(k)
                        # Count likes
                        counter += tweet.favorite_count
                        # Count retweets
                        counter += tweet.retweet_count
                        if len(df.iloc[:,3])!=0:
                            SST=counter/len(df.iloc[:,3])
                        else:
                            SST=0
                    SST_list.append(SST)
                    #count += result_count
                    #total_tweets += result_count
                    #print("Total # of Tweets added: ", total_tweets)
                    #print("-------------------")
                    time.sleep(5)
                
                #Since this is the final request, turn flag to false to move to the next time period.
                flag = False
                next_token = None
            time.sleep(5)
    


-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-03-01T00:00:00.000Z


In [26]:
SST_list

[212.51111111111112,
 410.3333333333333,
 16.907692307692308,
 631.5802469135803,
 1155.7619047619048,
 1155.7619047619048,
 31752.444444444445,
 122.24,
 78.8021978021978,
 1339.3555555555556,
 471.7464788732394,
 344.4642857142857,
 1016.7619047619048,
 489.4423076923077,
 229.50602409638554,
 37.67695961995249]

In [47]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time


os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAJm%2BcgEAAAAAdT6oZA5nHHlM%2BU07cRioA8QSXvI%3D1ZN1biqO1uHYO4GxiOEpSXCTElUu7fz7fjk8GTmEus6DJBYp2G'

def auth(): 
    return os.getenv('TOKEN')


def create_headers(bearer_token): 
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def append_to_csv(json_response, fileName):

    

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']#['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']
        
        if ('in_reply_to_user_id' in tweet):
            in_reply_to_user_id = tweet['in_reply_to_user_id']
        else:
            in_reply_to_user_id = " "
        
        

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']
        
        #test
        conversation_id= tweet['conversation_id']
        
      


        # 7. source
        #source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count,conversation_id,in_reply_to_user_id,text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

def tweets_to_dataframe(json_response):
    tweets = []
    for tweet in json_response['data']:
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']#['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']
        
        if ('in_reply_to_user_id' in tweet):
            in_reply_to_user_id = tweet['in_reply_to_user_id']
        else:
            in_reply_to_user_id = " "
        

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']
        
        #test
        conversation_id= tweet['conversation_id']
        

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a dictionary
        tweets.append({'author_id': author_id, 'created_at': created_at, 'geo': geo, 'tweet_id': tweet_id, 'lang': lang, 'like_count': like_count, 'quote_count': quote_count, 'reply_count': reply_count, 'retweet_count': retweet_count,'conversation_id':conversation_id,'in_reply_to_user_id':in_reply_to_user_id,'text':text})
    # Return the tweets as a dataframe
    return pd.DataFrame(tweets)
    
# append_to_csv(json_response, "data.csv") 

from datetime import timedelta
import datetime
def previous_datetime(datetime_str):
    # Convert the input string to a datetime object
    datetime_obj = datetime.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    # Subtract 2 days from the input datetime
    previous_datetime = datetime_obj - timedelta(days=2)
    # Return the previous datetime in the 'YYYY-MM-DD HH:MM:SS' format
    return previous_datetime.strftime('%Y-%m-%d %H:%M:%S')

from datetime import timedelta
def previous_tweet_datetime(created_at):
    # code to access the tweet using the tweet_id and retrieve the created_at value
    created_at = json_response['created_at']
    # Convert the created_at string to a datetime object
    datetime_obj = datetime.datetime.strptime(created_at, '%Y-%m-%d %H:%M:%S')
    # Subtract 1 second from the created_at datetime
    previous_datetime = datetime_obj - timedelta(seconds=1)
    # Return the previous datetime in the 'YYYY-MM-DD HH:MM:SS' format
    return previous_datetime.strftime('%Y-%m-%d %H:%M:%S')

import requests
import json
consumer_key
consumer_secret
def get_tweet_creation_date(tweet_id):
    # Set the API endpoint URL
    url = 'https://api.twitter.com/1.1/statuses/show.json'
    # Authenticate with the Twitter API using your API key and secret
    api_key=consumer_key
    api_secret=consumer_secret 
    bearer_token = api_key + ":" + api_secret
    headers = {'Authorization': 'Bearer ' + bearer_token}
    # Set the parameters for the API request
    params = {'id': tweet_id}
    # Send the API request
    response = requests.get(url, headers=headers, params=params)
    

#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword=["from:elonmusk","from:emmanuelmacron","from:marinelepen"]
start_list=["2022-04-22T00:00:00.000Z"]
end_list=["2022-04-24T00:00:00.000Z"] 
lang='en'
max_results = 500

total_tweets = 0

# Create file
csvFile = open("SSTdelta.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','conversation_id','in_reply_to_user_id','tweet'])
csvFile.close()
SST_liste=[]
for j in range(len(keyword)):
    res=keyword[j]
    for i in range(0,len(start_list)):

        num_file=str(j)

        # Inputs
        count = 0 # Counting tweets per time period
        max_count = 2000000 # Max tweets per time period
        flag = True
        next_token = None
        # Check if flag is true
        while flag:
            # Check if max_count reached
            if count >= max_count:
                break
            print("-------------------")
            print("Token: ", next_token)
            url = create_url(res, start_list[i],end_list[i], max_results)
            json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
            result_count = json_response['meta']['result_count']

            if 'next_token' in json_response['meta']:
                # Save the token to use for next call
                next_token = json_response['meta']['next_token']
                print("Next Token: ", next_token)
                if result_count is not None and result_count > 0 and next_token is not None:
                    print("Start Date: ", start_list[i])
                    append_to_csv(json_response, "C:/Users/srabh/Documents/PFEcode/SSTdelta.csv")
                    count += result_count
                    total_tweets += result_count
                    print("Total # of Tweets added: ", total_tweets)
                    print("-------------------")
                    time.sleep(5)                
            # If no next token exists
            else:
                if result_count is not None and result_count > 0:
                    print("-------------------")
                    print("Start Date: ", start_list[i])
                    #name_file="test_"+num_file+".csv"
                    df=tweets_to_dataframe(json_response)
                    for k in df.iloc[:,3]:
                        counter=0
                        tweet = api.get_status(k)
                        # Count likes
                        counter += tweet.favorite_count
                        # Count retweets
                        counter += tweet.retweet_count
                        if len(df.iloc[:,3])!=0:
                            SST=counter/len(df.iloc[:,3])
                        else:
                            SST=0
                    SST_liste.append(SST)
                    #count += result_count
                    #total_tweets += result_count
                    #print("Total # of Tweets added: ", total_tweets)
                    #print("-------------------")
                    time.sleep(5)
                
                #Since this is the final request, turn flag to false to move to the next time period.
                flag = False
                next_token = None
            time.sleep(5)
    print("Total number of results: ", total_tweets)



-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-04-22T00:00:00.000Z
Total number of results:  0
-------------------
Token:  None
Endpoint Response Code: 200
Total number of results:  0
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-04-22T00:00:00.000Z
Total number of results:  0


In [48]:
SST_liste

[3683.318181818182, 0.5]

In [ ]:
for i in range(len(name_file)):
    #df=pd.read_csv('C:/Users/srabh/AppData/Local/Programs/Microsoft VS Code/test.csv')
    df=pd.read_csv('test_'+str(i)+".csv")
    print(df)
    #df.iloc[:,3]

    for k in df.iloc[:,3]:
        # for user in tweepy.Paginator(client.get_retweeters,id=i, max_results=100).flatten(limit=100): 
        #     retweets_list_community.append(user.username)
        # time.sleep(20)
        # #get received likes
        # for user in tweepy.Paginator(client.get_liking_users,id=i, max_results=100).flatten(limit=100):
        #     like_list_community.append(user.username)
        # time.sleep(20)
        counter=0
        tweet = api.get_status(i)
        # Count likes
        counter += tweet.favorite_count
        # Count retweets
        counter += tweet.retweet_count
        if len(df.iloc[:,3])!=0:
            SST=counter/len(df.iloc[:,3])
        else:
            SST=0
        #return sum(counter[user_id] for user_id in user_id_list)

        


        


    


In [28]:
SST

1283.4545454545455

------------------------------------------------------------------------------------------------------------------------

In [ ]:
retweets_list_community = list(set(retweets_list_community))
like_list_community=list(set(like_list_community))

#retweets_list
sum_like_list=0
sum_RT_list=0
for i in like_list:
    if i in like_list_community or retweets_list_community:
        sum_like_list+=1
for j in retweets_list:
    if j in retweets_list_community or like_list_community:
        sum_RT_list+=1

        

In [2]:
def count_likes_and_retweets(tweet, user_id_list):
    count = 0
    user_id_set = set(user_id_list)
    for like, retweet in zip(tweet.favorite_count, tweet.retweet_count):
        if like.user.id in user_id_set:
            count += 1
        if retweet.user.id in user_id_set:
            count += 1
    return count

In [5]:
from collections import Counter

def count_likes_and_retweets(tweet, user_id_list):
    counter = Counter()
    for like in tweet.favorite_count:
        counter[like.user.id] += 1
    for retweet in tweet.retweet_count:
        counter[retweet.user.id] += 1
    return sum(counter[user_id] for user_id in user_id_list)

In [7]:
import tweepy

def count_likes_and_retweets(tweet_id, user_id_list):
    counter = Counter()
    consumer_key = "vILOcHxSRk9zOy5fZH3x7kIjS"
    consumer_secret = "VzPDu5BEq3x8VglxJPGKa1lFKfJ5OoKZfd90KOES1mdtMQsezo"
    access_token = "1526192300182978562-iHYctxjSI0k1cXZGIihYu6jQQJYZpF"
    access_token_secret = "zSwpEAjQwpDFL6jzhSrZZTf3spepUAxzTbvwy8TfNT1g9"
    # Authentication
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    # Get the tweet
    tweet = api.get_status(tweet_id)
    # Count likes
    for like in tweet.favorite_count:
        counter[like.user.id] += 1
    # Count retweets
    for retweet in tweet.retweet_count:
        counter[retweet.user.id] += 1
    return sum(counter[user_id] for user_id in user_id_list)

In [22]:
def count_likes_and_retweets(tweet_id, user_id_list):
    counter = Counter()
    # Authentication
    consumer_key = "vILOcHxSRk9zOy5fZH3x7kIjS"
    consumer_secret = "VzPDu5BEq3x8VglxJPGKa1lFKfJ5OoKZfd90KOES1mdtMQsezo"
    access_token = "1526192300182978562-iHYctxjSI0k1cXZGIihYu6jQQJYZpF"
    access_token_secret = "zSwpEAjQwpDFL6jzhSrZZTf3spepUAxzTbvwy8TfNT1g9"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    # Get the tweet
    tweet = api.get_status(tweet_id)
    # Count likes
    counter[tweet.user.id] += tweet.favorite_count
    print(tweet.favorite_count)
    # Count retweets
    counter[tweet.user.id] += tweet.retweet_count
    return counter
    #return sum(counter[user_id] for user_id in user_id_list)

In [23]:
a=count_likes_and_retweets("1518972397277396992",retweets_list)
a

417415


Counter({1212564056827031553: 489616})